# Lab 3-3: Model Optimization (ONNX & Quantization)

이 노트북에서는 **모델 최적화**를 대화형으로 실습합니다.

## 📋 실습 내용

1. 환경 설정 및 패키지 설치
2. Scikit-learn 모델 학습
3. ONNX 변환
4. 동적 양자화 적용
5. 성능 벤치마크
6. MLflow 메트릭 기록

---

## 1. 환경 설정

필요한 패키지를 설치하고 import합니다.

In [ ]:
# 패키지 설치
!pip install scikit-learn onnx onnxruntime skl2onnx mlflow joblib numpy -q

print("✅ 패키지 설치 완료!")

In [ ]:
import os
import time
import numpy as np
import joblib
import onnx
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Import 완료")
print(f"ONNX Runtime 버전: {ort.__version__}")

---

## 2. 데이터 로드 및 모델 학습

In [ ]:
# Iris 데이터셋 로드
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"📊 학습 데이터: {X_train.shape[0]}개")
print(f"📊 테스트 데이터: {X_test.shape[0]}개")
print(f"📊 피처 수: {X_train.shape[1]}")

In [ ]:
# RandomForest 모델 학습
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)
model.fit(X_train, y_train)

# 정확도 확인
train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))

print(f"📈 학습 정확도: {train_accuracy:.4f}")
print(f"📈 테스트 정확도: {test_accuracy:.4f}")

In [ ]:
# 출력 디렉토리 생성
os.makedirs('outputs', exist_ok=True)

# 원본 모델 저장
joblib.dump(model, 'outputs/model_original.joblib')
original_size = os.path.getsize('outputs/model_original.joblib') / 1024

print(f"💾 원본 모델 저장: outputs/model_original.joblib")
print(f"📦 모델 크기: {original_size:.2f} KB")

---

## 3. ONNX 변환

In [ ]:
def clean_onnx_opset(model):
    """ONNX 모델의 opset 중복 제거"""
    opset_dict = {}
    for opset in model.opset_import:
        domain = opset.domain if opset.domain else ""
        if domain not in opset_dict or opset.version > opset_dict[domain]:
            opset_dict[domain] = opset.version
    
    while len(model.opset_import) > 0:
        model.opset_import.pop()
    
    for domain, version in opset_dict.items():
        opset = model.opset_import.add()
        opset.domain = domain
        opset.version = version
    
    return model

In [ ]:
# ONNX 변환
print("🔄 Scikit-learn → ONNX 변환 중...")

initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(
    model, 
    initial_types=initial_type,
    target_opset=12,
    options={id(model): {'zipmap': False}}
)

# opset 정리
onnx_model = clean_onnx_opset(onnx_model)

# 유효성 검사
onnx.checker.check_model(onnx_model)
print("✅ ONNX 모델 유효성 검사 통과")

# 저장
onnx.save(onnx_model, 'outputs/model_optimized.onnx')
onnx_size = os.path.getsize('outputs/model_optimized.onnx') / 1024

print(f"💾 ONNX 모델 저장: outputs/model_optimized.onnx")
print(f"📦 ONNX 모델 크기: {onnx_size:.2f} KB")
print(f"📉 크기 감소: {(1 - onnx_size/original_size)*100:.1f}%")

In [ ]:
# ONNX 모델 정보 확인
print("📋 ONNX 모델 opset 정보:")
for opset in onnx_model.opset_import:
    domain = opset.domain if opset.domain else "ai.onnx"
    print(f"   - Domain: {domain}, Version: {opset.version}")

In [ ]:
# 예측 검증
X_test_float32 = X_test.astype(np.float32)

# 원본 모델 예측
original_pred = model.predict(X_test)

# ONNX 모델 예측
onnx_session = ort.InferenceSession('outputs/model_optimized.onnx')
input_name = onnx_session.get_inputs()[0].name
onnx_output = onnx_session.run(None, {input_name: X_test_float32})
onnx_pred = onnx_output[0]

if len(onnx_pred.shape) > 1 and onnx_pred.shape[1] > 1:
    onnx_pred = np.argmax(onnx_pred, axis=1)

# 비교
match_rate = np.sum(original_pred == onnx_pred) / len(original_pred) * 100
print(f"🎯 예측 일치율: {match_rate:.1f}%")

---

## 4. 양자화 (Quantization)

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

print("🔄 동적 양자화 적용 중...")

try:
    quantize_dynamic(
        model_input='outputs/model_optimized.onnx',
        model_output='outputs/model_quantized.onnx',
        weight_type=QuantType.QUInt8,
    )
    print("✅ 양자화 완료")
except Exception as e:
    print(f"⚠️ QUInt8 실패, QInt8로 재시도: {e}")
    quantize_dynamic(
        model_input='outputs/model_optimized.onnx',
        model_output='outputs/model_quantized.onnx',
        weight_type=QuantType.QInt8,
    )
    print("✅ 양자화 완료 (QInt8)")

quant_size = os.path.getsize('outputs/model_quantized.onnx') / 1024
print(f"📦 양자화 모델 크기: {quant_size:.2f} KB")

In [ ]:
# 양자화 모델 정확도 검증
quant_session = ort.InferenceSession('outputs/model_quantized.onnx')
quant_input_name = quant_session.get_inputs()[0].name
quant_output = quant_session.run(None, {quant_input_name: X_test_float32})
quant_pred = quant_output[0]

if len(quant_pred.shape) > 1 and quant_pred.shape[1] > 1:
    quant_pred = np.argmax(quant_pred, axis=1)

quant_accuracy = accuracy_score(y_test, quant_pred)
print(f"📈 양자화 모델 정확도: {quant_accuracy:.4f} ({quant_accuracy*100:.2f}%)")

---

## 5. 성능 벤치마크

In [ ]:
def benchmark_inference(predict_fn, X_test, n_iterations=1000):
    """추론 성능 벤치마크"""
    # 워밍업
    for _ in range(10):
        predict_fn(X_test)
    
    # 벤치마크
    times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        predict_fn(X_test)
        end = time.perf_counter()
        times.append((end - start) * 1000)
    
    return np.mean(times)

In [ ]:
n_iterations = 1000
print(f"⏱️ 각 모델 {n_iterations}회 추론 수행 중...")

# 원본 모델
original_time = benchmark_inference(
    lambda x: model.predict(x), X_test, n_iterations
)

# ONNX 모델
onnx_time = benchmark_inference(
    lambda x: onnx_session.run(None, {input_name: x.astype(np.float32)})[0],
    X_test, n_iterations
)

# 양자화 모델
quant_time = benchmark_inference(
    lambda x: quant_session.run(None, {quant_input_name: x.astype(np.float32)})[0],
    X_test, n_iterations
)

# 속도 향상 계산
onnx_speedup = original_time / onnx_time
quant_speedup = original_time / quant_time

print(f"\n⏱️ 추론 시간 (평균):")
print(f"   • 원본 sklearn: {original_time:.4f} ms (1.0x)")
print(f"   • ONNX:         {onnx_time:.4f} ms ({onnx_speedup:.1f}x)")
print(f"   • 양자화:       {quant_time:.4f} ms ({quant_speedup:.1f}x)")

---

## 6. 결과 요약

In [ ]:
print("\n" + "="*65)
print("                     벤치마크 결과 요약")
print("="*65)
print(f"{'모델':<15} | {'크기 (KB)':>10} | {'추론 (ms)':>10} | {'속도향상':>8} | {'정확도':>8}")
print("-"*65)
print(f"{'원본 sklearn':<15} | {original_size:>10.2f} | {original_time:>10.4f} | {'1.0x':>8} | {test_accuracy*100:>7.2f}%")
print(f"{'ONNX':<15} | {onnx_size:>10.2f} | {onnx_time:>10.4f} | {onnx_speedup:>7.1f}x | {accuracy_score(y_test, onnx_pred)*100:>7.2f}%")
print(f"{'양자화':<15} | {quant_size:>10.2f} | {quant_time:>10.4f} | {quant_speedup:>7.1f}x | {quant_accuracy*100:>7.2f}%")
print("="*65)

print("\n🎯 핵심 성과:")
print(f"   • 모델 크기: {original_size:.1f} KB → {quant_size:.1f} KB ({(1-quant_size/original_size)*100:.1f}% 감소)")
print(f"   • 추론 속도: {original_time:.2f} ms → {quant_time:.2f} ms ({quant_speedup:.1f}x 향상)")
print(f"   • 정확도 유지: {quant_accuracy*100:.2f}%")

---

## 7. MLflow 기록 (선택)

In [ ]:
# MLflow Tracking URI 설정
import mlflow

MLFLOW_URI = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000"
mlflow.set_tracking_uri(MLFLOW_URI)
print(f"🔗 MLflow Tracking URI: {MLFLOW_URI}")

In [ ]:
# MLflow 실험 및 Run 기록
experiment_name = "lab3-3-model-optimization"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="notebook-benchmark"):
    # 크기 메트릭
    mlflow.log_metric("original_size_kb", original_size)
    mlflow.log_metric("onnx_size_kb", onnx_size)
    mlflow.log_metric("quantized_size_kb", quant_size)
    
    # 속도 메트릭
    mlflow.log_metric("original_inference_ms", original_time)
    mlflow.log_metric("onnx_inference_ms", onnx_time)
    mlflow.log_metric("quantized_inference_ms", quant_time)
    mlflow.log_metric("onnx_speedup", onnx_speedup)
    mlflow.log_metric("quantized_speedup", quant_speedup)
    
    # 정확도 메트릭
    mlflow.log_metric("original_accuracy", test_accuracy)
    mlflow.log_metric("quantized_accuracy", quant_accuracy)
    
    # 파라미터
    mlflow.log_param("n_iterations", n_iterations)
    mlflow.log_param("quantization_type", "dynamic_uint8")
    
    # 아티팩트
    mlflow.log_artifact('outputs/model_optimized.onnx')
    mlflow.log_artifact('outputs/model_quantized.onnx')
    
    print(f"✅ MLflow 기록 완료!")
    print(f"   Run ID: {mlflow.active_run().info.run_id}")
    print(f"   실험: {experiment_name}")

---

## ✅ 실습 완료!

### 학습한 내용
- ONNX 포맷으로 모델 변환
- 동적 양자화 적용
- 추론 성능 벤치마크
- MLflow 실험 추적

### 다음 단계
- Lab 3-2 E2E Pipeline에 모델 최적화 적용
- KServe로 최적화된 모델 배포
- Prometheus/Grafana로 추론 성능 모니터링